# Calculate Well Trajectory from File Directory

Accept any file or group of files and convert them to well trajectory objects

In [1]:
# ONLY RUN ONCE, must restart notebook otherwise
import pandas as pd
import os
from pathlib import Path
current_dir = Path.cwd()
path = current_dir.parent
print(path)
#change directory to get src class data
os.chdir(path)
from src.data_object import *
from src.transforms import *

from src.calculable_object import *
from src.wellbore_trajectory import *

C:\Users\BpAmos\Documents\repos\directional-survey-converter


## Get Files

**transforms.py**

get a list of data files from a folder or group of folders.

This can be recusive, use extensions, anything. Returns a list of PathLib paths

In [2]:
file_paths = get_files(path, folders='data')
file_paths.items

[WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.json'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey_many.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_interpolate_data.csv')]

In [3]:
my_json = file_paths.items[2]
my_json

WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey_many.csv')

In [4]:
my_json.suffix
my_json.stem

'wellbore_survey_many'

In [9]:
from typing import Union, Collection
PathOrStr = Union[Path,str]
# TODO: maybe use kwargs instead of all the input params?
def from_csv(path:PathOrStr, wellId_name: Optional[str] = None, md_name: Optional[str] = None,
            inc_name: Optional[str] = None, azim_name: Optional[str] = None,
            surface_latitude_name: Optional[str] = None,
            surface_longitude_name: Optional[str] = None,
            surface_x_name: Optional[str] = None,
            surface_y_name: Optional[str] = None):
    df = pd.read_csv(path, sep=',')

    
    return from_df(df, wellId_name = wellId_name, md_name = md_name,
            inc_name = inc_name, azim_name = azim_name,
            surface_latitude_name = surface_latitude_name,
            surface_longitude_name = surface_longitude_name,
            surface_x_name = surface_x_name,
            surface_y_name = surface_y_name)

In [10]:

from typing import Union, Collection
IntsOrStrs = Union[int, Collection[int], str, Collection[str]]
def is_listy(x:Any)->bool: return isinstance(x, (tuple,list))
def df_names_to_idx(names:IntsOrStrs, df:DataFrame):
    "Return the column indexes of `names` in `df`."
    if not is_listy(names): names = [names]
    if isinstance(names[0], int): return names
    return [df.columns.get_loc(c) for c in names]

In [11]:
def from_df(df, wellId_name: str = None, md_name: str = None,
            inc_name: str = None, azim_name: str = None,
            surface_latitude_name: Optional[str] = None,
            surface_longitude_name: Optional[str] = None,
            surface_x_name: Optional[str] = None,
            surface_y_name: Optional[str] = None):
    """
    convert a well survey df into a list of dicts format used in `WellboreTrajectory`
    User must specify column names for wellId, md, inc, azim, and either both 
    surface_latitude, surface_longitude, or both surface_x, surface_y
    
    :Parameters:
    df
    wellId_name
    md_name
    inc_name
    azim_name
    surface_latitude_name
    surface_longitude_name
    surface_x_name
    surface_y_name
    
    :Return: 
    list of dicts
    
    """
    # if no column names are specified use 0:5 for dir survey obj
    surface_latitude_name = if_none(surface_latitude_name, None)
    surface_longitude_name = if_none(surface_longitude_name, None)
    surface_x_name = if_none(surface_x_name, None)
    surface_y_name = if_none(surface_y_name, None)
    
    # check to make sure no columns have NaN values
    inputs = df.iloc[:,df_names_to_idx(list(df.columns), df)]
    assert not inputs.isna().any().any(), f"You have NaN values in column(s) {cols} of your dataframe, please fix it."
    
    #group by wellId, ensures this will work with single well or mulitple.
    grouped = df.groupby(wellId_name)
    
    #initialize empty dict and list
    d={}
    dlist=[]
    # loop through groups converting them to the proper dict format
    for name, group in grouped:
        group.reset_index(inplace=True, drop=True)
    
        if surface_latitude_name is not None and surface_longitude_name is not None:
            dataclass_obj = dict(data= dict(wellId=group[wellId_name][0],
                                 md=np.array(group[md_name]),
                                 inc=np.array(group[inc_name]),
                                 azim=np.array(group[azim_name]),
                                 surface_latitude=group[surface_latitude_name][0],
                                 surface_longitude=group[surface_longitude_name][0]))

        if surface_x_name is not None and surface_y_name is not None:
            dataclass_obj = dict(wellId=df[wellId_name][0],
                                 md=np.array(df[md_name]),
                                 inc=np.array(df[inc_name]),
                                 azim=np.array(df[azim_name]),
                                 surface_x=df[surface_x_name][0],
                                 surface_y=df[surface_y_name][0])
        
        d=dataclass_obj
        dlist.append(d)
    return dlist

In [12]:
obj = from_csv(my_json, wellId_name = 'wellId', md_name='md', inc_name='inc', azim_name='azim', 
              surface_latitude_name='surface_latitude', surface_longitude_name='surface_longitude')
obj

[{'data': {'wellId': 'well_A',
   'md': array([    0.    ,    36.75  ,   813.5505,   840.    ,   945.    ,
           1050.    ,  1155.    ,  1260.    ,  1365.    ,  1470.    ,
           1575.    ,  1680.    ,  1785.    ,  1890.    ,  1995.    ,
           2100.    ,  2205.    ,  2310.    ,  2415.    ,  2520.    ,
           2625.    ,  2730.    ,  2835.    ,  2940.    ,  3045.    ,
           3150.    ,  3255.    ,  3360.    ,  3465.    ,  3570.    ,
           3675.    ,  3780.    ,  3885.    ,  3990.    ,  4095.    ,
           4200.    ,  4305.    ,  4410.    ,  4515.    ,  4620.    ,
           4725.    ,  4830.    ,  4935.    ,  5040.    ,  5145.    ,
           5250.    ,  5355.    ,  5460.    ,  5565.    ,  5670.    ,
           5775.    ,  5880.    ,  5985.    ,  6090.    ,  6195.    ,
           6300.    ,  6405.    ,  6510.    ,  6615.    ,  6720.    ,
           6773.2035,  6858.6   ,  6956.25  ,  7056.    ,  7153.65  ,
           7254.45  ,  7352.1   ,  7452.9   ,  7550.5

In [15]:
data = obj[0]

In [244]:
class WellboreTrajectory2(CalculableObject):

    def __init__(self, data=None):
        """
        DirectionalSurvey object with a wells directional survey info

        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """
        self.data = data
        self.deviation_survey_obj = DeviationSurvey(**self.data[0]['data'])
        #self.deviation_survey_obj = DeviationSurvey(**self.data[0]['data'])
    
    @classmethod
    def from_json(cls,path:PathOrStr):
        
        with open(path) as json_file:
            json_data = json.load(json_file)
        json_file.close()
        
        data = dict(data = dict(json_data))
        
        
        res = cls(data=[data])
        return res
    
    @classmethod
    def from_csv(cls,path:PathOrStr, wellId_name: Optional[str] = None, md_name: Optional[str] = None,
        inc_name: Optional[str] = None, azim_name: Optional[str] = None,
        surface_latitude_name: Optional[str] = None,
        surface_longitude_name: Optional[str] = None,
        surface_x_name: Optional[str] = None,
        surface_y_name: Optional[str] = None):
        
        
        
        
        df = pd.read_csv(path, sep=',')


        return cls(from_df(df, wellId_name = wellId_name, md_name = md_name,
            inc_name = inc_name, azim_name = azim_name,
            surface_latitude_name = surface_latitude_name,
            surface_longitude_name = surface_longitude_name,
            surface_x_name = surface_x_name,
            surface_y_name = surface_y_name))
    
    @classmethod
    def from_df(cls, df, wellId_name: str = None, md_name: str = None,
            inc_name: str = None, azim_name: str = None,
            surface_latitude_name: Optional[str] = None,
            surface_longitude_name: Optional[str] = None,
            surface_x_name: Optional[str] = None,
            surface_y_name: Optional[str] = None):
        """
        convert a well survey df into a list of dicts format used in `WellboreTrajectory`
        User must specify column names for wellId, md, inc, azim, and either both 
        surface_latitude, surface_longitude, or both surface_x, surface_y

        :Parameters:
        df
        wellId_name
        md_name
        inc_name
        azim_name
        surface_latitude_name
        surface_longitude_name
        surface_x_name
        surface_y_name

        :Return: 
        list of dicts

        """
        # if no column names are specified use 0:5 for dir survey obj
        surface_latitude_name = if_none(surface_latitude_name, None)
        surface_longitude_name = if_none(surface_longitude_name, None)
        surface_x_name = if_none(surface_x_name, None)
        surface_y_name = if_none(surface_y_name, None)

        # check to make sure no columns have NaN values
        inputs = df.iloc[:,df_names_to_idx(list(df.columns), df)]
        assert not inputs.isna().any().any(), f"You have NaN values in column(s) {cols} of your dataframe, please fix it."

        #group by wellId, ensures this will work with single well or mulitple.
        grouped = df.groupby(wellId_name)

        #initialize empty dict and list
        d={}
        dlist=[]
        # loop through groups converting them to the proper dict format
        for name, group in grouped:
            group.reset_index(inplace=True, drop=True)

            if surface_latitude_name is not None and surface_longitude_name is not None:
                dataclass_obj = dict(data= dict(wellId=group[wellId_name][0],
                                     md=np.array(group[md_name]),
                                     inc=np.array(group[inc_name]),
                                     azim=np.array(group[azim_name]),
                                     surface_latitude=group[surface_latitude_name][0],
                                     surface_longitude=group[surface_longitude_name][0]))

            if surface_x_name is not None and surface_y_name is not None:
                dataclass_obj = dict(data= dict(wellId=df[wellId_name][0],
                                     md=np.array(df[md_name]),
                                     inc=np.array(df[inc_name]),
                                     azim=np.array(df[azim_name]),
                                     surface_x=df[surface_x_name][0],
                                     surface_y=df[surface_y_name][0]))

            d=dataclass_obj
            dlist.append(d)
            
        res = cls(data=dlist)
        return res
    
    
    
    def calculate_survey_points(self, **kwargs):
        #print(self.deviation_survey_obj)
        super().calculate_survey_points(**kwargs)
        

In [264]:
#file_path = file_paths.items[2]
file_path = file_paths.items[0]
file_path = file_paths.items[1]
file_path

WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.json')

In [265]:
#my_well1 = WellboreTrajectory2()
#my_well1.data

In [266]:
my_well = WellboreTrajectory2.from_json(file_path)
#my_well.data
#my_well.deviation_survey_obj

my_well.calculate_survey_points()
my_well.deviation_survey_obj

DeviationSurvey(wellId='well_C', md=array([    0.  ,    35.  ,   774.81,   800.  ,   900.  ,  1000.  ,
        1100.  ,  1200.  ,  1300.  ,  1400.  ,  1500.  ,  1600.  ,
        1700.  ,  1800.  ,  1900.  ,  2000.  ,  2100.  ,  2200.  ,
        2300.  ,  2400.  ,  2500.  ,  2600.  ,  2700.  ,  2800.  ,
        2900.  ,  3000.  ,  3100.  ,  3200.  ,  3300.  ,  3400.  ,
        3500.  ,  3600.  ,  3700.  ,  3800.  ,  3900.  ,  4000.  ,
        4100.  ,  4200.  ,  4300.  ,  4400.  ,  4500.  ,  4600.  ,
        4700.  ,  4800.  ,  4900.  ,  5000.  ,  5100.  ,  5200.  ,
        5300.  ,  5400.  ,  5500.  ,  5600.  ,  5700.  ,  5800.  ,
        5900.  ,  6000.  ,  6100.  ,  6200.  ,  6300.  ,  6400.  ,
        6450.67,  6532.  ,  6625.  ,  6720.  ,  6813.  ,  6909.  ,
        7002.  ,  7098.  ,  7191.  ,  7286.  ,  7379.  ,  7475.  ,
        7569.  ,  7663.  ,  7758.  ,  7851.  ,  7947.  ,  8040.  ,
        8135.  ,  8229.  ,  8324.  ,  8418.  ,  8512.  ,  8606.  ,
        8700.  ,  8794.  ,

In [261]:
file_path = file_paths.items[2]
#file_path = file_paths.items[0]
print(file_path)

my_well = WellboreTrajectory2.from_csv(file_path,  wellId_name = 'wellId', md_name='md', inc_name='inc', azim_name='azim', 
              surface_latitude_name='surface_latitude', surface_longitude_name='surface_longitude')
#my_well.data
#my_well.deviation_survey_obj


C:\Users\BpAmos\Documents\repos\directional-survey-converter\data\wellbore_survey_many.csv


In [262]:
my_well.calculate_survey_points()
my_well.deviation_survey_obj

DeviationSurvey(wellId='well_A', md=array([    0.    ,    36.75  ,   813.5505,   840.    ,   945.    ,
        1050.    ,  1155.    ,  1260.    ,  1365.    ,  1470.    ,
        1575.    ,  1680.    ,  1785.    ,  1890.    ,  1995.    ,
        2100.    ,  2205.    ,  2310.    ,  2415.    ,  2520.    ,
        2625.    ,  2730.    ,  2835.    ,  2940.    ,  3045.    ,
        3150.    ,  3255.    ,  3360.    ,  3465.    ,  3570.    ,
        3675.    ,  3780.    ,  3885.    ,  3990.    ,  4095.    ,
        4200.    ,  4305.    ,  4410.    ,  4515.    ,  4620.    ,
        4725.    ,  4830.    ,  4935.    ,  5040.    ,  5145.    ,
        5250.    ,  5355.    ,  5460.    ,  5565.    ,  5670.    ,
        5775.    ,  5880.    ,  5985.    ,  6090.    ,  6195.    ,
        6300.    ,  6405.    ,  6510.    ,  6615.    ,  6720.    ,
        6773.2035,  6858.6   ,  6956.25  ,  7056.    ,  7153.65  ,
        7254.45  ,  7352.1   ,  7452.9   ,  7550.55  ,  7650.3   ,
        7747.95  ,  7848.7

In [148]:
my_well = WellboreTrajectory2.from_csv(file_path,  wellId_name = 'wellId', md_name='md', inc_name='inc', azim_name='azim', 
              surface_latitude_name='surface_latitude', surface_longitude_name='surface_longitude')
my_well.data
#my_well.wellId_name

KeyError: 'wellId'

In [14]:
# use that path to greate a Deviation Survey Object
my_calc_obj = WellboreTrajectory(data)
#my_calc_obj.deviation_survey_obj
# Calcualate Survey Points from Object
my_calc_obj.calculate_survey_points()
my_calc_obj.deviation_survey_obj

DeviationSurvey(wellId='well_A', md=array([    0.    ,    36.75  ,   813.5505,   840.    ,   945.    ,
        1050.    ,  1155.    ,  1260.    ,  1365.    ,  1470.    ,
        1575.    ,  1680.    ,  1785.    ,  1890.    ,  1995.    ,
        2100.    ,  2205.    ,  2310.    ,  2415.    ,  2520.    ,
        2625.    ,  2730.    ,  2835.    ,  2940.    ,  3045.    ,
        3150.    ,  3255.    ,  3360.    ,  3465.    ,  3570.    ,
        3675.    ,  3780.    ,  3885.    ,  3990.    ,  4095.    ,
        4200.    ,  4305.    ,  4410.    ,  4515.    ,  4620.    ,
        4725.    ,  4830.    ,  4935.    ,  5040.    ,  5145.    ,
        5250.    ,  5355.    ,  5460.    ,  5565.    ,  5670.    ,
        5775.    ,  5880.    ,  5985.    ,  6090.    ,  6195.    ,
        6300.    ,  6405.    ,  6510.    ,  6615.    ,  6720.    ,
        6773.2035,  6858.6   ,  6956.25  ,  7056.    ,  7153.65  ,
        7254.45  ,  7352.1   ,  7452.9   ,  7550.55  ,  7650.3   ,
        7747.95  ,  7848.7

In [228]:
class DataObject(metaclass=abc.ABCMeta):
    """
    A abstract base class to work with subclasses `DeviationSurvey` and `CalculableObject`.

    """

    @abstractmethod
    def validate(self):
        pass

    @abstractmethod
    def serialize(self):
        pass

    @abstractmethod
    def deserialize(self):
        pass


In [229]:

class CalculableObject(DataObject):

    def __init__(self, deviation_survey_obj, **kwargs):
        """
        DirectionalSurvey object with a wells directional survey info

        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """

        self.deviation_survey_obj = DeviationSurvey(**deviation_survey_obj)

    def deserialize(self):
        super().deserialize()

    def validate(self):
        super().validate()

    def serialize(self):
        super().serialize()

    #@classmethod
    def crs_transform(self, crs_in: str):
        crs_out = "EPSG:4326"

        x = self.deviation_survey_obj.surface_x
        y = self.deviation_survey_obj.surface_y
        self.deviation_survey_obj.surface_latitude, self.deviation_survey_obj.surface_longitude = \
            crs_transformer(crs_out=crs_out, crs_in=crs_in, x=x, y=y)

    def minimum_curvature_algo(self):
        """
        Calculate values along the wellbore using only provided md, inc, and azim
        calculate TVD, n_s_deviation, e_w_deviation, and dls

        :parameter:
        None

        :return:
        tvd_cum:            (np.array)
        dls:                (np.array)
        e_w_deviation:      (np.array)
        n_s_deviation:      (np.array)
        """
        # Following are the calculations for Minimum Curvature Method

        md = self.deviation_survey_obj.md
        inc = self.deviation_survey_obj.inc
        azim = self.deviation_survey_obj.azim

        # Convert to Radians
        inc_rad = np.multiply(inc, 0.0174533)
        azim_rad = np.multiply(azim, 0.0174533)

        # Shift all array values +1
        md_shift = shift(md, 1, cval=np.NaN)
        inc_rad_shift = shift(inc_rad, 1, cval=np.NaN)
        azim_rad_shift = shift(azim_rad, 1, cval=np.NaN)

        # calculate beta (dog leg angle)
        beta = np.arccos(
            np.cos(inc_rad - inc_rad_shift - (np.sin(inc_rad_shift) *
                                              np.sin(inc_rad) *
                                              (1 - np.cos(azim_rad - azim_rad_shift))
                                              )))

        # convert first nan value to 0
        beta[np.isnan(beta)] = 0

        # dog leg severity per 100 ft
        dls = (beta * 57.2958 * 100) / (md - md_shift)
        dls[np.isnan(dls)] = 0

        # calculate ratio factor (radians)
        # replace 0 for rf calc
        beta_no_zero = np.where(beta == 0, 1, beta)
        rf = np.where(beta == 0, 1, 2 / beta_no_zero * np.tan(beta_no_zero / 2))

        # calculate total vertical depth
        tvd = ((md - md_shift) / 2) * (np.cos(inc_rad_shift) + np.cos(inc_rad)) * rf
        tvd[np.isnan(tvd)] = 0

        tvd = np.cumsum(tvd, dtype=float)

        # calculating NS
        ns = ((md - md_shift) / 2) * (
                np.sin(inc_rad_shift) * np.cos(azim_rad_shift) +
                np.sin(inc_rad) * np.cos(azim_rad)) * rf
        ns[np.isnan(ns)] = 0

        n_s_deviation = np.cumsum(ns, dtype=float)

        # calculating EW
        ew = ((md - md_shift) / 2) * (
                np.sin(inc_rad_shift) * np.sin(azim_rad_shift) +
                np.sin(inc_rad) * np.sin(azim_rad)) * rf
        ew[np.isnan(ew)] = 0

        e_w_deviation = np.cumsum(ew, dtype=float)

        self.deviation_survey_obj.tvd = tvd
        self.deviation_survey_obj.dls = dls
        self.deviation_survey_obj.e_w_deviation = e_w_deviation
        self.deviation_survey_obj.n_s_deviation = n_s_deviation

    def calculate_lat_lon_from_deviation_points(self):
        """
        get lat lon points from survey using minimum curvature algorithm generated values
        for the ns and ew deviations

        :parameter:
        e_w_deviation:          (np.array)
        n_s_deviation:          (np.array)

        required survey data:
        self.surface_latitude:  (float)
        self.surface_longitude: (float)

        :return:
        Calculated attributes for lat lon points
        longitude_points:       (np.array)
        latitude_points:        (np.array)
        zone_number:            (str)
        zone_letter:            (str)
        x_points:               (np.array)
        y_points:               (np.array)
        surface_x:              (np.array)
        surface_y:              (np.array)
        """
        surface_latitude = self.deviation_survey_obj.surface_latitude
        surface_longitude = self.deviation_survey_obj.surface_longitude
        e_w_deviation = self.deviation_survey_obj.e_w_deviation
        n_s_deviation = self.deviation_survey_obj.n_s_deviation

        # create X and Y deviation points and zone number and letter
        surface_x, surface_y, zone_number, zone_letter = utm.from_latlon(surface_latitude, surface_longitude)

        # add the x and y offset from the surface x and y for each point * meters conversion
        x_points = np.multiply(e_w_deviation, 0.3048) + surface_x
        y_points = np.multiply(n_s_deviation, 0.3048) + surface_y

        # create lat lon points along the wellbore from the x,y,zone number and leter
        latitude_points, longitude_points = utm.to_latlon(x_points, y_points, zone_number, zone_letter)

        self.deviation_survey_obj.longitude_points = longitude_points
        self.deviation_survey_obj.latitude_points = latitude_points
        self.deviation_survey_obj.zone_number = zone_number
        self.deviation_survey_obj.zone_letter = zone_letter
        self.deviation_survey_obj.x_points = x_points
        self.deviation_survey_obj.y_points = y_points
        self.deviation_survey_obj.surface_x = surface_x
        self.deviation_survey_obj.surface_y = surface_y

    def calculate_horizontal(self,  horizontal_angle: Optional[float] = 88.0):
        """
        calculate if the inclination of the wellbore is in its horizontal section
        If the wellbore inclination is greater than 88 degrees the wellbore is horizontal
        else the well is vertical

        :parameter:
        None

        :return:
        inc_hz:     (np.array)
        """
        # get inc points
        inc = self.deviation_survey_obj.inc

        # inc greater than 88 deg is horizontal, else vertical
        inc_hz = np.greater(inc, horizontal_angle)
        inc_hz = np.where((inc_hz == True), 'Horizontal', 'Vertical')

        self.deviation_survey_obj.isHorizontal = inc_hz

    @abstractmethod
    def calculate_survey_points(self, **kwargs):
        """
        Run the minimum_curvature_algo, calculate_lat_lon_from_deviation_points, and calculate_horizontal
        functions to calculate the wells lat lon points and other attributes from provided md, inc, azim
        and surface lat lon

        :parameter:
        None

        :return:
        survey_points_obj:       (DirectionalSurvey obj)

        """
        for k,v in kwargs.items():
            print(k,v)

        if self.deviation_survey_obj.surface_latitude is None and self.deviation_survey_obj.surface_longitude is None:
            self.crs_transform(**kwargs)

        # get minimum curvature points
        self.minimum_curvature_algo()

        # get lat lon points
        self.calculate_lat_lon_from_deviation_points()

        # calc horizontal
        self.calculate_horizontal()

In [12]:
file_paths = get_files(path, folders='data')
file_paths.items

[WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.json'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey_many.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_interpolate_data.csv')]

In [297]:
class TransformData():

    #def __init__(self):
    def __init__(self, path:PathOrStr='.', items=None):
        """
        DirectionalSurvey object with a wells directional survey info

        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """
        self.path = path
        self.items = items

        
        
#         self.deserialized_obj = get_from_json(path)
#         self.deviation_survey_obj = DeviationSurvey(**self.deserialized_obj)

    @classmethod
    def get_data(cls, path:PathOrStr, folder:str=None, extensions:str=None, recurse:bool=True, **kwargs):
        """Create an `ItemList` in `path` from the filenames that have a suffix in `extensions`.
        `recurse` determines if we search subfolders."""
        path = Path(path)
        assert path.is_dir() and path.exists(), f"{path} is not a valid directory."
        
        #print(path)
        #items = cls(get_files(path, folder=folder, extensions=extensions, recurse=recurse))
        items = cls(get_files(path, folder, extensions))
        #print(items.path)
        return items

    def deserialize(self):
        with open(self.path) as json_file:
            data = json.load(json_file)
        json_file.close()
        return data
    
    


In [298]:
str(path)

'C:\\Users\\BpAmos\\Documents\\repos\\directional-survey-converter'

In [299]:
my_items = TransformData().get_data(str(path), folder='data')
my_items.path

[]

In [153]:
my_thing = WellboreTrajectory2()

my_thing.path


'.'

In [156]:
file_paths = get_files(my_thing.path, extensions='.csv')
file_paths.items

[WindowsPath('data/wellbore_survey.csv'),
 WindowsPath('data/wellbore_survey_many.csv'),
 WindowsPath('data/well_interpolate_data.csv')]

In [ ]:
def get_file_type(file_path):
    suffix = file_path.suffix
    if suffix is ".json":
        data = get_from_json(file_path)
        print('yes json')
    else:
        if suffix is ".csv":
            
            #convert to df
            #convert to json

In [8]:
class WellboreTrajectory2():

    def __init__(self, path, file_type):
        """
        DirectionalSurvey object with a wells directional survey info

        Attributes:
        directional_survey_points (Dataclass Object) DataObject object
        """
        self.path = path
        self.file_type = file_type
        #self.deserialized_obj = get_from_json(path)
        #self.deviation_survey_obj = DeviationSurvey(**self.deserialized_obj)

In [5]:
with open(my_json) as json_file:
    data = json.load(json_file)
json_file.close()
data

{'wellId': 'well_C',
 'md': [0.0,
  35.0,
  774.81,
  800.0,
  900.0,
  1000.0,
  1100.0,
  1200.0,
  1300.0,
  1400.0,
  1500.0,
  1600.0,
  1700.0,
  1800.0,
  1900.0,
  2000.0,
  2100.0,
  2200.0,
  2300.0,
  2400.0,
  2500.0,
  2600.0,
  2700.0,
  2800.0,
  2900.0,
  3000.0,
  3100.0,
  3200.0,
  3300.0,
  3400.0,
  3500.0,
  3600.0,
  3700.0,
  3800.0,
  3900.0,
  4000.0,
  4100.0,
  4200.0,
  4300.0,
  4400.0,
  4500.0,
  4600.0,
  4700.0,
  4800.0,
  4900.0,
  5000.0,
  5100.0,
  5200.0,
  5300.0,
  5400.0,
  5500.0,
  5600.0,
  5700.0,
  5800.0,
  5900.0,
  6000.0,
  6100.0,
  6200.0,
  6300.0,
  6400.0,
  6450.67,
  6532.0,
  6625.0,
  6720.0,
  6813.0,
  6909.0,
  7002.0,
  7098.0,
  7191.0,
  7286.0,
  7379.0,
  7475.0,
  7569.0,
  7663.0,
  7758.0,
  7851.0,
  7947.0,
  8040.0,
  8135.0,
  8229.0,
  8324.0,
  8418.0,
  8512.0,
  8606.0,
  8700.0,
  8794.0,
  8859.0,
  8924.0,
  9018.0,
  9112.0,
  9206.0,
  9292.0,
  9386.0,
  9481.0,
  9574.0,
  9669.0,
  9764.0,
  9858.0,

In [7]:
# use that path to greate a Deviation Survey Object
dev = WellboreTrajectory(data)
#my_calc_obj.deviation_survey_obj
# Calcualate Survey Points from Object
my_calc_obj.calculate_survey_points()
my_calc_obj.deviation_survey_obj

In [8]:
# Calcualate Survey Points from Object
my_calc_obj.calculate_survey_points()
my_calc_obj.deviation_survey_obj

DeviationSurvey(wellId='well_C', md=array([    0.  ,    35.  ,   774.81,   800.  ,   900.  ,  1000.  ,
        1100.  ,  1200.  ,  1300.  ,  1400.  ,  1500.  ,  1600.  ,
        1700.  ,  1800.  ,  1900.  ,  2000.  ,  2100.  ,  2200.  ,
        2300.  ,  2400.  ,  2500.  ,  2600.  ,  2700.  ,  2800.  ,
        2900.  ,  3000.  ,  3100.  ,  3200.  ,  3300.  ,  3400.  ,
        3500.  ,  3600.  ,  3700.  ,  3800.  ,  3900.  ,  4000.  ,
        4100.  ,  4200.  ,  4300.  ,  4400.  ,  4500.  ,  4600.  ,
        4700.  ,  4800.  ,  4900.  ,  5000.  ,  5100.  ,  5200.  ,
        5300.  ,  5400.  ,  5500.  ,  5600.  ,  5700.  ,  5800.  ,
        5900.  ,  6000.  ,  6100.  ,  6200.  ,  6300.  ,  6400.  ,
        6450.67,  6532.  ,  6625.  ,  6720.  ,  6813.  ,  6909.  ,
        7002.  ,  7098.  ,  7191.  ,  7286.  ,  7379.  ,  7475.  ,
        7569.  ,  7663.  ,  7758.  ,  7851.  ,  7947.  ,  8040.  ,
        8135.  ,  8229.  ,  8324.  ,  8418.  ,  8512.  ,  8606.  ,
        8700.  ,  8794.  ,

## Get Wellbore Trajectory object

In [10]:
class WellboreTrajectory3(CalculableObject):

    

    
    def deserialize(self):
        with open(self) as json_file:
            data = json.load(json_file)
        json_file.close()
        return data

#     def calculate_survey_points(self):
#         super().calculate_survey_points()

In [11]:
my_survey = WellboreTrajectory2().deserialize(my_json)
my_survey

TypeError: __init__() missing 1 required positional argument: 'deviation_survey_obj'

In [12]:
my_survey = WellboreTrajectory2(my_survey)
my_survey.calculate_survey_points()
my_survey.deviation_survey_obj

TypeError: ABCMeta object argument after ** must be a mapping, not WellboreTrajectory2